In [31]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import mysql.connector


db = mysql.connector.connect(host = "localhost",
                             username = "root",
                             password = "aman@373",
                             database = "ecom")

cur = db.cursor()
                            
                        

In [32]:
#List all unique cities where customers are located


query = """ select distinct customer_city from customers """

cur.execute(query)

data = cur.fetchall()
df = pd.DataFrame(data, columns = ["customer city" " avarage product/order",])
df.head(10)


,customer city avarage product/order
0,franca
1,sao bernardo do campo
2,sao paulo
3,mogi das cruzes
4,campinas
5,jaragua do sul
6,timoteo
7,curitiba
8,belo horizonte
9,montes claros


#Count the number of oder of city in 2017

In [33]:
query = """ select count(order_id) from orders where year(order_purchase_timestamp) = 2017 """

cur.execute(query)

data = cur.fetchall()

data

[(225505,)]

#Find the total sales per category

In [34]:
query = """ select upper (products.product_category) category,
round(sum(payments.payment_value),2) sales
from products join order_items
on products.product_id = order_items.product_id
join payments
on payments.order_id = order_items.order_id
group by category """

cur.execute(query)

data = cur.fetchall()



df = pd.DataFrame(data, columns = ["category", "sales"])
df

,category,sales
0,SPORT LEISURE,5568510.24
1,HOME APPLIANCES,382129.08
2,FURNITURE OFFICE,2587305.95
3,COOL STUFF,3118792.00
4,BED TABLE BATH,6850214.68
...,...,...
69,PARTY ARTICLES,23865.24
70,LA CUISINE,11654.12
71,FASHION CHILDREN'S CLOTHING,3142.68
72,INSURANCE AND SERVICES,1298.04


#Calculate the persentage of order that were paid in installments.

In [35]:
query = """ select (sum(case when payment_installments >= 1 then 1 else 0 end)) / count(*)*100 from payments
"""

cur.execute(query)

data = cur.fetchall()

data

[(Decimal('99.9981'),)]

#count the number of customers from each state

In [36]:
query = """ select customer_state, count(customer_id)
from customers group by customer_state
"""

cur.execute(query)

data = cur.fetchall()
data




[('SP', 250476),
 ('SC', 21822),
 ('MG', 69810),
 ('PR', 30270),
 ('RJ', 77112),
 ('RS', 32796),
 ('PA', 5850),
 ('GO', 12120),
 ('ES', 12198),
 ('BA', 20280),
 ('MA', 4482),
 ('MS', 4290),
 ('CE', 8016),
 ('DF', 12840),
 ('RN', 2910),
 ('PE', 9912),
 ('MT', 5442),
 ('AM', 888),
 ('AP', 408),
 ('AL', 2478),
 ('RO', 1518),
 ('PB', 3216),
 ('TO', 1680),
 ('PI', 2970),
 ('AC', 486),
 ('SE', 2100),
 ('RR', 276)]

#Calculate the number order per month in 2018

In [37]:
query = """ select monthname(order_purchase_timestamp) months, count(order_id) order_count from orders where year(order_purchase_timestamp) = 2018
group by months
"""

cur.execute(query)

data = cur.fetchall()

data

[('July', 31460),
 ('August', 32560),
 ('February', 33640),
 ('June', 30835),
 ('March', 36055),
 ('January', 36345),
 ('May', 34365),
 ('April', 34695),
 ('September', 80),
 ('October', 20)]

In [39]:
#Find the avarage number of product per order, group by the city

query = """ with count_per_order as
( select orders.order_id, orders.customer_id, count(order_items.order_id) as oc
from orders join order_items
on orders.order_id = order_items.order_id
group by orders.order_id, orders.customer_id)


select customers.customer_city, round(avg(count_per_order.oc),2) average_orders
from customers join count_per_order
on customers.customer_id = count_per_order.customer_id
group by customers.customer_city

"""

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ["customer city", " avarage product/order"])
df.head(10)


,customer city,avarage product/order
0,treze tilias,6.36
1,indaial,5.58
2,sao jose dos campos,5.69
3,sao paulo,5.78
4,porto alegre,5.87
5,santos,5.81
6,sao francisco do sul,6.18
7,sao vicente,5.45
8,joinville,5.51
9,sao vendelino,5.00


#Calcutale the percentage of total revenue contributed by each product category

In [40]:
query = """ select upper (products.product_category) category,
round((sum(payments.payment_value)/ (select sum(payment_value) from payments))*100,2) sales_percentage
from products join order_items
on products.product_id = order_items.product_id
join payments
on payments.order_id = order_items.order_id
group by category  order by sales_percentage desc"""


cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["category", "percentage distribution"])
df

,category,percentage distribution
0,BED TABLE BATH,42.79
1,HEALTH BEAUTY,41.41
2,COMPUTER ACCESSORIES,39.61
3,FURNITURE DECORATION,35.73
4,WATCHES PRESENT,35.71
...,...,...
69,PC GAMER,0.05
70,HOUSE COMFORT 2,0.04
71,CDS MUSIC DVDS,0.03
72,FASHION CHILDREN'S CLOTHING,0.02


#Identify the corelation  between product price and the number of times a products has been purchase

In [41]:

query = """SELECT products.product_category,
count(order_items.product_id),
round(avg(order_items.price),2)
from products join order_items
on products.product_id = order_items.product_id
group by products.product_category"""

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["category", "order count", "price"])
df

arr1 = df["order count"]
arr2 = df["price"]

a = np.corrcoef([arr1,arr2])
print("The corretalion of between price and number of  time a produts has been purchase", a [0][1])


The corretalion of between price and number of  time a produts has been purchase -0.10631514167157564


#Calculate the total revenue genrated by the each seller, and rank them by revenue

In [43]:


query = """select *, dense_rank() over(order by revenue desc) as rn from
(select order_items.seller_id, sum(payments.payment_value) 
revenue from order_items join payments
on order_items.order_id = payments.order_id
group by order_items.seller_id) as a """

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["seller_id", "revenue", "rank"])
df 

,seller_id,revenue,rank
0,7c67e1448b00f6e969d365cea6b010ab,507166.907302,1
1,1025f0e2d44d7041d6cf58b6550e0bfa,308222.039840,2
2,4a3ca9315b744ce9f8e9374361493884,301245.269765,3
3,1f50f920176fa81dab994f9023523100,290253.420128,4
4,53243585a1d6dc2643021fd1853d8905,284903.080498,5
...,...,...,...
3090,ad14615bdd492b01b0d97922e87cb87f,19.209999,3082
3091,702835e4b785b67a084280efca355756,18.559999,3083
3092,4965a7002cca77301c82d3f91b82e1a9,16.360001,3084
3093,77128dec4bec4878c37ab7d6169d6f26,15.220000,3085


#Calculate the moving avarage of order value of each customer over thaire order history

In [44]:
query = """select customer_id, order_purchase_timestamp, payment,
avg(payment) over(partition by customer_id order by order_purchase_timestamp
rows between 2 preceding and current row) as mov_avg
from
(select orders.customer_id, orders.order_purchase_timestamp,
payments.payment_value as payment
from payments join orders
on payments.order_id = orders.order_id) as a """


cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data)
df


,0,1,2,3
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
1,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
2,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
3,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
4,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
...,...,...,...,...
519425,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001
519426,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001
519427,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001
519428,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001


#Calculate the cumulative sales per months by each year

In [45]:
query = """Select years, months, payment, sum(payment)
over(order by years, months) cumulative_sales from 
(select year(orders.order_purchase_timestamp) as years,
month(orders.order_purchase_timestamp) as months,
round(sum(payments.payment_value),2 ) as payment from orders join payments
on orders.order_id = payments.order_id
group by years, months order by years, months) as a

"""

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data)
df


,0,1,2,3
0,2016,9,1261.20,1261.20
1,2016,10,295452.40,296713.60
2,2016,12,98.10,296811.70
3,2017,1,692440.20,989251.90
4,2017,2,1459540.05,2448791.95
5,2017,3,2249318.00,4698109.95
6,2017,4,2088940.15,6787050.10
7,2017,5,2964594.10,9751644.20
8,2017,6,2556381.90,12308026.10
9,2017,7,2961914.60,15269940.70


#Calculate the year over year grouth rate of total sales

In [46]:
query = """ with a as(select year(orders.order_purchase_timestamp) as years,
round(sum(payments.payment_value),2) as payment from orders join payments
on orders.order_id = payments.order_id
group by years order by years)

select years, ((payment - lag(payment, 1) over(order by years))/
lag(payment,1) over(order by years))* 100 from a

"""

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["year", "yoy % grow"])
df


,year,yoy % grow
0,2016,NaN
1,2017,12112.703758
2,2018,20.000924


#Identify the top 3 customers who spend most money in each year 

In [50]:
query = """ select years ,customer_id, payment, d_rank
from
(select year(orders.order_purchase_timestamp) years,
orders.customer_id,
sum(payments.payment_value) payment,
dense_rank() over (partition by year(orders.order_purchase_timestamp)
order by sum(payments.payment_value) desc) d_rank
from orders join payments 
on payments.order_id = orders.order_id
group by year(orders.order_purchase_timestamp),
orders.customer_id) as a 
where d_rank <= 3

"""
cur.execute(query)
data = cur.fetchall()
data

[(2016, 'a9dc96b027d1252bbac0a9b72d837fc6', 7117.750244140625, 1),
 (2016, '1d34ed25963d5aae4cf3d7f3a4cda173', 7003.699951171875, 2),
 (2016, '4a06381959b6670756de02e07b83815f', 6138.900146484375, 3),
 (2017, '1617b1357756262bfa56ab541c47bc16', 68320.400390625, 1),
 (2017, 'c6e2731c5b391845f6800c97401a43a9', 34646.55029296875, 2),
 (2017, '3fd6777bbce08a352fddd04e4a7cc8f6', 33633.30078125, 3),
 (2018, 'ec5b2ba62e574342386871631fafd3fc', 36374.3994140625, 1),
 (2018, 'f48d464a0baaea338cb25f816991ab1f', 34611.0498046875, 2),
 (2018, 'e0a2412720e9ea4f26c1ac985f6a7358', 24047.19970703125, 3)]